In [ ]:
import os
import rootutils
import torch

# adding root to python path
rootutils.setup_root(
    os.path.abspath(''), indicator=['.git', 'pyproject.toml'], pythonpath=True
)

from src.models.components.cnn_cam_multihead import CNNCAMMultihead
from src.models.components.vit_rollout_multihead import VitRolloutMultihead
from src.models.components.nn_utils import weight_load

#### CNN export

In [ ]:
cnn_model = CNNCAMMultihead(
    backbone='torchvision.models/efficientnet_v2_s',
    return_node='features.6.0.block.0',
    multi_head=True,
)
weights = weight_load(
    ckpt_path='../trained_models/models--DeepVisionXplain--efficientnet_v2_s_downscaled_pcb/',
    weights_only=True,
)
cnn_model.load_state_dict(weights)
cnn_model.eval()
x = torch.randn((1, 3, 224, 224))
torch.onnx.export(
    cnn_model,
    x,
    'efficientnet_v2_s_downscaled_pcb.onnx',
    export_params=True,
    opset_version=12,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['map', 'output'],
    dynamic_axes={
        'input': {0: 'batch_size'},
        'map': {0: 'batch_size'},
        'output': {0: 'batch_size'},
    },
)

#### ViT export

In [ ]:
vit_model = VitRolloutMultihead(
    backbone='timm/vit_tiny_patch16_224.augreg_in21k_ft_in1k', multi_head=True
)
weights = weight_load(
    ckpt_path='../trained_models/models--DeepVisionXplain--vit_tiny_patch16_224.augreg_in21k_ft_in1k_pcb/',
    weights_only=True,
)
vit_model.load_state_dict(weights)
vit_model.eval()
x = torch.randn((1, 3, 224, 224))
torch.onnx.export(
    vit_model,
    x,
    'vit_tiny_patch16_224.augreg_in21k_ft_in1k_pcb.onnx',
    export_params=True,
    opset_version=12,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['map', 'output'],
    dynamic_axes={
        'input': {0: 'batch_size'},
        'map': {0: 'batch_size'},
        'output': {0: 'batch_size'},
    },
)